![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# The output

Your query should return a table in the following format:
| industry  | year | num\_unicorns       | average\_valuation\_billions |
| --------- | ---- | ------------------- | ---------------------------- |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.

In [1]:
SELECT 
i.industry as industry,
count(d.*) as total_unicorns
from dates as d
join industries as i on i.company_id = d.company_id
where extract(year from date_joined) in (2019,2020,2021)
group by industry
order by total_unicorns desc
limit 3


,industry,total_unicorns
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


In [2]:
Select industry,
extract(year from d.date_joined) as year,
count(d.company_id) as num_unicorn,
Round(avg(f.valuation)/1000000000,2) as average_valuation_billions
from industries as i
join dates as d on i.company_id = d.company_id
join funding as f on f.company_id =i.company_id
group by 1,2

,industry,year,num_unicorn,average_valuation_billions
0,Mobile & telecommunications,2017,5,2.80
1,Internet software & services,2015,4,1.25
2,Fintech,2018,10,8.60
3,Mobile & telecommunications,2019,4,2.00
4,Artificial intelligence,2012,1,2.00
...,...,...,...,...
119,E-commerce & direct-to-consumer,2022,7,1.57
120,Other,2015,2,1.00
121,Consumer & retail,2017,4,10.50
122,Fintech,2019,20,6.80


In [3]:
with top_industries as (SELECT 
i.industry as industry,
count(d.*) as total_unicorns
from dates as d
join industries as i on i.company_id = d.company_id
where extract(year from date_joined) in (2019,2020,2021)
group by industry
order by total_unicorns desc
limit 3),

Yearly_industry_info as (Select industry,
extract(year from d.date_joined) as year,
count(d.company_id) as num_unicorns,
Round(avg(f.valuation)/1000000000,2) as average_valuation_billions
from industries as i
join dates as d on i.company_id = d.company_id
join funding as f on f.company_id =i.company_id
group by 1,2)

SELECT *
from Yearly_industry_info
where year in (2019,2020,2021) AND industry in (SELECT 
industry from top_industries)
order by year desc, num_unicorns desc

,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Internet software & services,2021,119,2.15
2,E-commerce & direct-to-consumer,2021,47,2.47
3,Internet software & services,2020,20,4.35
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Fintech,2020,15,4.33
6,Fintech,2019,20,6.80
7,Internet software & services,2019,13,4.23
8,E-commerce & direct-to-consumer,2019,12,2.58
